### Set path to original pyNeuroChem. Please change to your own path

#### Import required libraries

In [1]:
import sys
#sys.path.append('/home/olexandr/notebooks/ASE_ANI/lib')
from ase_interface import ANIENS
from ase_interface import aniensloader
from ase_interface import ANIENS,ensemblemolecule

In [2]:
import numpy as np
import  ase
import time
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md.verlet import VelocityVerlet
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS 

#0.0252235

#### Load the benzene molecule from the xyz

In [3]:
bz = read('/home/jujuman/Gits/ASE_ANI/examples/data/bz.xyz')

#### Set ANI ensemble calculator

In [4]:
# Current ANI model options are:
# '../ani_models/ani-1ccx_8x.info' Coupled cluster transfer learned model
# '../ani_models/ani-1x_8x.info'   Full ANI-1x wb97x/6-31g* dataset model
bz.set_calculator(ANIENS(aniensloader('../ani_models/ani-1ccx_8x.info',0,multigpu=False)))

In [5]:
# Bias - parabola
weight_f=200.0
mult_const_f = 1.0
const_f = 0.8

weight_e=0.0
mult_const_e = 1.0
const_e = 0.14

def bias_Efunc(sigma_f):
    #print("sigma "+str(sigma))
    return weight_f * (np.power((mult_const_f*sigma_f-const_f),2))

# Force bias function
def bias_Ffunc(sigma_f, dsigma):
    #print(sigma_e)
    return -weight_f *2*(mult_const_f*sigma_f-const_f)*mult_const_f* dsigma

# Set the bias potential
bz.calc.set_sigmabias(bias_Efunc, bias_Ffunc, epsilon=0.001, disable_ani=False)

In [6]:
Fn = bz.calc.calculate_numerical_forces(bz, d=0.001)

!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disabled!
!ANI disab

In [7]:
bz.get_potential_energy()
Fa = bz.get_forces()

!ANI disabled!
!ANI disabled!


In [8]:
np.mean(np.abs(Fn-Fa))

0.15298406203490889

In [9]:
np.mean(np.sum(Fa,axis=0))

0.0036703024661835504

In [10]:
np.mean(np.sum(Fn,axis=0))

0.0031181799992433903

In [11]:
Fa

array([[ 7.71753351e-02, -5.08433895e-01, -8.56483095e-02],
       [-1.58531727e-01,  4.01634824e-01,  2.97741904e-01],
       [-3.86430982e-03,  3.61772251e-01, -9.55308068e-02],
       [-8.43971601e-02, -1.11241443e-01,  2.27677533e-01],
       [-6.10687651e-02,  5.95349192e-01, -1.26825381e-01],
       [ 1.44246450e-01, -4.49146830e-01, -1.73323798e-01],
       [-2.09126074e-01, -3.84913766e+00,  2.59256784e+00],
       [ 1.17838090e+00, -4.43401045e+00, -2.07093352e+00],
       [-1.33910213e+00,  2.35227983e-01,  4.66308167e+00],
       [-1.11205414e+00,  4.36472211e+00,  1.90573032e+00],
       [ 2.20166677e-01,  3.64762340e+00, -2.52755558e+00],
       [ 1.29739451e+00, -2.65194201e-01, -4.53435581e+00]])

In [12]:
Fn

array([[ 1.34828897e-02,  7.53163931e-05, -1.27518101e-02],
       [-1.71881034e-02,  2.75226057e-04,  1.26668162e-02],
       [ 1.39555909e-02, -9.18555565e-03,  1.59417575e-02],
       [ 1.71302313e-02, -6.51394384e-03, -1.40051779e-02],
       [-1.03398970e-02,  6.53552547e-03,  1.77864788e-02],
       [-1.27673200e-02,  1.49428547e-02, -1.81995943e-02],
       [-2.20795595e-01, -4.04670989e+00,  2.68493060e+00],
       [ 1.11019139e+00, -4.34263897e+00, -1.94004482e+00],
       [-1.33174848e+00,  2.76354355e-01,  4.71360914e+00],
       [-1.11303233e+00,  4.34925672e+00,  1.94269341e+00],
       [ 2.20367242e-01,  4.04246572e+00, -2.68599496e+00],
       [ 1.33179173e+00, -2.76508681e-01, -4.71667332e+00]])

In [ ]:
-0.21560924*2

#### Helper functions

In [ ]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    return np.arccos(np.clip(np.dot(unit_vector(v1), unit_vector(v2)), -1.0, 1.0))

#### Run Optimization

In [ ]:
# X = bz.get_positions()
# eqdist = np.linalg.norm(X[0]-X[1])
# eqenrg = bz.get_potential_energy()
# eqangl = angle_between(X[0]-X[1],X[0]-X[2])
# print('Distances: ',np.array([np.linalg.norm(X[0]-X[1]),
#                     np.linalg.norm(X[0]-X[2]),
#                     np.linalg.norm(X[0]-X[3]),
#                     np.linalg.norm(X[0]-X[4])]))
# print('Angles: ',np.array([57.2958*angle_between(X[0]-X[1],X[0]-X[2]),
#                  57.2958*angle_between(X[0]-X[1],X[0]-X[3]),
#                  57.2958*angle_between(X[0]-X[1],X[0]-X[4]),
#                  57.2958*angle_between(X[0]-X[2],X[0]-X[3]),
#                  57.2958*angle_between(X[0]-X[2],X[0]-X[4]),
#                  57.2958*angle_between(X[0]-X[3],X[0]-X[4])]))

# start_time = time.time()
# dyn = LBFGS(bz)
# dyn.run(fmax=0.001)
# print('[ANI Total time:', time.time() - start_time, 'seconds]')

# X = bz.get_positions()
# eqdist = np.linalg.norm(X[0]-X[1])
# eqenrg = bz.get_potential_energy()
# eqangl = angle_between(X[0]-X[1],X[0]-X[2])
# print('Distances: ',np.array([np.linalg.norm(X[0]-X[1]),
#                     np.linalg.norm(X[0]-X[2]),
#                     np.linalg.norm(X[0]-X[3]),
#                     np.linalg.norm(X[0]-X[4])]))
# print('Angles: ',np.array([57.2958*angle_between(X[0]-X[1],X[0]-X[2]),
#                  57.2958*angle_between(X[0]-X[1],X[0]-X[3]),
#                  57.2958*angle_between(X[0]-X[1],X[0]-X[4]),
#                  57.2958*angle_between(X[0]-X[2],X[0]-X[3]),
#                  57.2958*angle_between(X[0]-X[2],X[0]-X[4]),
#                  57.2958*angle_between(X[0]-X[3],X[0]-X[4])]))

#### Setup dynamics

In [ ]:
# Temperature
T = 300.0

# Time step
dt = 1.0

# We want to run MD with constant energy using the Langevin algorithm
# with a time step of 5 fs, the temperature T and the friction
# coefficient to 0.02 atomic units.
#dyn = Langevin(bz, dt * units.fs, T * units.kB, 0.01)
dyn = VelocityVerlet(bz, dt=0.5 * units.fs)

#### Setup printer function

In [ ]:
CHdist = []
CHHang = []
energy = []
temper = []
toteng = []

def printenergy(a=bz,b=dyn,c=CHdist,d=CHHang,e=energy,t=temper,et=toteng):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Step: %i Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.7feV' % (dyn.get_number_of_steps(),epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
    X = a.get_positions()
    
    c.append([dt*dyn.get_number_of_steps(),np.linalg.norm(X[0]-X[1]),np.linalg.norm(X[0]-X[2]),np.linalg.norm(X[0]-X[3]),np.linalg.norm(X[0]-X[4])])
    d.append([dt*dyn.get_number_of_steps(),angle_between(X[0]-X[1],X[0]-X[2]),angle_between(X[0]-X[1],X[0]-X[3]),angle_between(X[0]-X[1],X[0]-X[4]),angle_between(X[0]-X[2],X[0]-X[3]),angle_between(X[0]-X[2],X[0]-X[4]),angle_between(X[0]-X[3],X[0]-X[4])])
    e.append([dt*dyn.get_number_of_steps(),a.get_potential_energy()])
    t.append([dt*dyn.get_number_of_steps(),ekin / (1.5 * units.kB)])
    et.append([dt*dyn.get_number_of_steps(),epot + ekin])
    
    
dyn.attach(printenergy, interval=25)
printenergy()

#### Run 2000 steps of MD

In [ ]:
start_time = time.time()
dyn.run(2000) # Do 1ps of MD
print('[ANI Total time:', time.time() - start_time, 'seconds]')

CHdist = np.vstack(CHdist)
CHHang = np.vstack(CHHang)
temper = np.vstack(temper)
energy = np.vstack(energy)
toteng = np.vstack(toteng)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=300)
plt.plot(CHdist[:,0]/1000,CHdist[:,1],label="Distance 1",alpha=0.5)
plt.plot(CHdist[:,0]/1000,CHdist[:,2],label="Distance 2",alpha=0.5)
plt.plot(CHdist[:,0]/1000,CHdist[:,3],label="Distance 3",alpha=0.5)
plt.plot(CHdist[:,0]/1000,CHdist[:,4],label="Distance 4",alpha=0.5)
#plt.hlines(eqdist,0,(CHdist[:,0]/1000).max(),linestyle='--',zorder=100)

plt.legend()

plt.xlabel("Simulation Time (ps)")
plt.ylabel(r"C-H Distance ($\AA$)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=300)
plt.hist(CHdist[:,1],bins=50,label="Distance 1",alpha=0.5)
plt.hist(CHdist[:,2],bins=50,label="Distance 2",alpha=0.5)
plt.hist(CHdist[:,3],bins=50,label="Distance 3",alpha=0.5)
plt.hist(CHdist[:,4],bins=50,label="Distance 4",alpha=0.5)
#plt.vlines(eqdist,0,700,linestyle='--',zorder=100)

plt.legend()

plt.ylabel("Count")
plt.xlabel(r"C-H Distance ($\AA$)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=300)
plt.plot(CHHang[:,0]/1000,57.2958*CHHang[:,1],label="Angle 1",alpha=0.5)
plt.plot(CHHang[:,0]/1000,57.2958*CHHang[:,2],label="Angle 2",alpha=0.5)
plt.plot(CHHang[:,0]/1000,57.2958*CHHang[:,3],label="Angle 3",alpha=0.5)
plt.plot(CHHang[:,0]/1000,57.2958*CHHang[:,4],label="Angle 4",alpha=0.5)
plt.plot(CHHang[:,0]/1000,57.2958*CHHang[:,5],label="Angle 5",alpha=0.5)
plt.plot(CHHang[:,0]/1000,57.2958*CHHang[:,6],label="Angle 6",alpha=0.5)

#plt.hlines(57.2958*eqangl,0,(CHHang[:,0]/1000).max(),linestyle='--',zorder=100)

plt.legend()

plt.xlabel("Simulation Time (ps)")
plt.ylabel(r"H-C-H Angle (Degrees)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=300)
plt.hist(57.2958*CHHang[:,1],bins=50,label="Angle 1",alpha=0.5)
plt.hist(57.2958*CHHang[:,2],bins=50,label="Angle 2",alpha=0.5)
plt.hist(57.2958*CHHang[:,3],bins=50,label="Angle 3",alpha=0.5)
plt.hist(57.2958*CHHang[:,4],bins=50,label="Angle 4",alpha=0.5)
plt.hist(57.2958*CHHang[:,5],bins=50,label="Angle 5",alpha=0.5)
plt.hist(57.2958*CHHang[:,6],bins=50,label="Angle 6",alpha=0.5)


#plt.vlines(57.2958*eqangl,0,670,linestyle='--',zorder=100)

plt.legend()

plt.ylabel("Count")
plt.xlabel(r"H-C-H Angle (Degrees)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=300)
plt.plot(energy[:,0]/1000,energy[:,1])
#plt.hlines(energy,0,(energy[:,0]/1000).max(),linestyle='--',zorder=100)

plt.xlabel("Simulation Time (ps)")
plt.ylabel(r"Potential energy (eV)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=300)
plt.plot(temper[:,0]/1000,temper[:,1])
plt.hlines(300.0,0,(temper[:,0]/1000).max(),linestyle='--',zorder=100)

plt.xlabel("Simulation Time (ps)")
plt.ylabel(r"Temperature (K)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=300)
plt.plot(toteng[:,0]/1000,toteng[:,1])
#plt.hlines(toteng,0,(toteng[:,0]/1000).max(),linestyle='--',zorder=100)

plt.xlabel("Simulation Time (ps)")
plt.ylabel(r"Total E energy (eV)")
plt.show()